In [ ]:
from sklearn.cluster import MiniBatchDPMeans, DPMeans,KMeans, MiniBatchKMeans, DBSCAN, MeanShift, AgglomerativeClustering, OPTICS
from evaluations import *
# from dpmmpython.dpmmwrapper import DPMMPython
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import black_box as bb
from sklearn.metrics import normalized_mutual_info_score as nmi
from time import time
from tqdm import tqdm
from torchvision.datasets import MNIST
from sklearn.decomposition import PCA

In [ ]:
EXP_NAME = "ImageNet100"
data = np.array(pd.read_csv('./data/imagenet_short.csv',header=None))
embeddings = data[:,:64].astype(np.float32)
gt = data[:,64].astype(int)
D = embeddings.shape[1]
N = embeddings.shape[0]
K_count=100
perm = np.random.permutation(len(gt))
data = embeddings[perm]
gt = gt[perm]
(X_train,y_train),(X_val,y_val),(X_test,y_test) = train_val_test_split(data,gt,0.001,0.1)

In [ ]:
np.save('opt_res/x_train_im.npy',X_train)
np.save('opt_res/y_train_im.npy',y_train)
np.save('opt_res/x_test_im.npy',X_test)
np.save('opt_res/y_test_im.npy',y_test)

In [ ]:
X_train = np.load('opt_res/x_train_im.npy')
y_train = np.load('opt_res/y_train_im.npy')
X_test = np.load('opt_res/x_test_im.npy')
y_test = np.load('opt_res/y_test_im.npy')

### DBSCAN

In [ ]:
def search_dbscan(pars):    
    labels = DBSCAN(eps=pars[0], min_samples=int(np.round(pars[1]))).fit_predict(X_test)
    return -nmi(labels,y_test)
                                                 

In [ ]:
best_params = bb.search_min(f = search_dbscan,  # given function
                            domain = [  # ranges of each parameter
                                [0.001, 100.],
                                [1., 10.]
                                ],
                            budget = 400,  # total number of function calls available
                            batch = 16,
                            resfile = 'opt_res/dbscan_im.csv')

In [ ]:
params = np.array([2.67954056, 1.38473185])
# timings =[]
# nmis = []
# ks =[]
for i in tqdm(range(2)):
    tic = time()
    labels = DBSCAN(eps=params[0], min_samples=int(np.round(params[1]))).fit_predict(X_train)
    toc = time()-tic
    timings.append(toc)
    nmis.append(nmi(labels,y_train))
    ks.append(len(np.unique(labels)))
    print(timings[-1],nmis[-1],ks[-1])

In [ ]:
print(np.mean(timings))
print(np.std(timings))
print(np.mean(nmis))
print(np.std(nmis))
print(np.mean(ks))
print(np.std(ks))

In [ ]:
def search_meanshit(pars):    
    labels = MeanShift(bandwidth=pars[0]).fit_predict(X_test)
    return -nmi(labels,y_test)

In [ ]:
best_params = bb.search_min(f = search_meanshit,  # given function
                            domain = [  # ranges of each parameter
                                [0.1, 10.],
                                ],
                            budget = 20,  # total number of function calls available
                            batch = 10,
                            resfile = 'opt_res/meanshift.csv')

In [ ]:
best_params = np.array([1.2685]) #Evaluated seperetly due to bb optimize multi process bug with notebooks.

In [ ]:
params = best_params

In [ ]:
timings =[]
nmis = []
ks =[]
for i in tqdm(range(3)):
    tic = time()
    labels = MeanShift(bandwidth=params[0]).fit_predict(X_train)
    toc = time()-tic
    timings.append(toc)
    nmis.append(nmi(labels,y_train))
    ks.append(len(np.unique(labels)))
    print(timings[-1],nmis[-1],ks[-1])

In [ ]:
print(np.mean(timings))
print(np.std(timings))
print(np.mean(nmis))
print(np.std(nmis))
print(np.mean(ks))
print(np.std(ks))

### P-DP-Means

In [ ]:
def search_dpmeans(pars):    
    labels = DPMeans(delta=pars[0]).fit_predict(X_test)
    return -nmi(labels,y_test)

In [ ]:
best_params = bb.search_min(f = search_dpmeans,  # given function
                            domain = [  # ranges of each parameter
                                [1, 100.],
                                ],
                            budget = 100,  # total number of function calls available
                            batch = 16,
                            resfile = 'opt_res/pdpmeans_im.csv')

In [ ]:
best_params = np.array([85.69745234])
timings =[]
nmis = []
ks =[]
for i in tqdm(range(3)):
    tic = time()
    labels = DPMeans(delta=best_params[0],n_init=1).fit_predict(X_train)
    toc = time()-tic
    timings.append(toc)
    nmis.append(nmi(labels,y_train))
    ks.append(len(np.unique(labels)))
    print(timings[-1],nmis[-1],ks[-1])

In [ ]:
print(np.mean(timings))
print(np.std(timings))
print(np.mean(nmis))
print(np.std(nmis))
print(np.mean(ks))
print(np.std(ks))

### MiniBatch PDC-DP-Means

In [ ]:
def search_dpmeans(pars):    
    labels = MiniBatchDPMeans(delta=pars[0], batch_size= int(np.round(pars[1]))).fit_predict(X_test)
    return -nmi(labels,y_test)

In [ ]:
best_params = bb.search_min(f = search_dpmeans,  # given function
                            domain = [  # ranges of each parameter
                                [0.1, 20.],
                                [10,2000]],
                            budget = 200,  # total number of function calls available
                            batch = 16,
                            resfile = 'opt_res/mbpdpmeans.csv')

In [ ]:
best_params = np.array([10.52243421, 10.00000611])

In [ ]:
search_dpmeans(best_params)

In [ ]:
best_params = np.array([115.61347713, 213.675161001])
timings =[]
nmis = []
ks =[]
for i in tqdm(range(3)):
    tic = time()
    labels = MiniBatchDPMeans(delta=best_params[0], batch_size= int(np.round(best_params[1]))).fit_predict(X_train)
    toc = time()-tic
    timings.append(toc)
    nmis.append(nmi(labels,y_train))
    ks.append(len(np.unique(labels)))
    print(timings[-1],nmis[-1],ks[-1])

In [ ]:
print(np.mean(timings))
print(np.std(timings))
print(np.mean(nmis))
print(np.std(nmis))
print(np.mean(ks))
print(np.std(ks))